# Install requirements (dependencies)

In [ ]:
!pip install keras_tuner

In [ ]:
!pip install tensorflow-gpu

In [ ]:
#!nvidia-smi

# Import Libraries

In [ ]:
from tensorflow import keras
from keras_tuner import Hyperband
from tensorflow.keras import callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#download kaggle api (kaggle.json) and import it here
#from google.colab import files
#files.upload()

In [ ]:
# 600 permissions: only the owner has full read and write access to the API key (chmod=change mode)

#!mkdir ~/.kaggle
#!cp /content/kaggle.json ~/.kaggle/
#!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#!kaggle competitions download -c cassava-disease -f train.zip
#!kaggle competitions download -c cassava-disease -f test.zip

In [ ]:
#!unzip train.zip -d /content/drive/MyDrive/ #/dev/null
#!unzip test.zip > /dev/null

In [ ]:
train_path = '/content/drive/MyDrive/train'

In [ ]:
image_data_generator = ImageDataGenerator(rescale=1./255,
                                          rotation_range=90, 
                                          shear_range=0.2, 
                                          zoom_range=0.2, 
                                          horizontal_flip=True, 
                                          vertical_flip=True,
                                          validation_split=0.2
                                          )
                                                                               
train_set = image_data_generator.flow_from_directory(train_path, 
                                                     subset='training', 
                                                     target_size=(224,224), 
                                                     class_mode='categorical', 
                                                     batch_size=32, 
                                                     shuffle=True,
                                                     interpolation='nearest',
                                                     color_mode="rgb",
                                                     )
val_set = image_data_generator.flow_from_directory(train_path, 
                                                   subset='validation', 
                                                   target_size=(224,224), 
                                                   class_mode='categorical', 
                                                   batch_size=32, 
                                                   shuffle=False,
                                                   interpolation='nearest',  
                                                   color_mode="rgb"
                                                   )

Found 5656 images belonging to 5 classes.
Found 1129 images belonging to 5 classes.


### Define a hypermodel through two approaches:

+ By using a model builder function
+ By subclassing the HyperModel class of the Keras Tuner API
Two pre-defined HyperModel classes - <mark>HyperXception and HyperResNet</mark> can also be used for computer vision applications.

In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  #keras.applications.MobileNetV3Large() 
  
  model.add(keras.layers.InputLayer(input_shape=(224, 224, 3)))
  #model.add(keras.layers.Flatten())

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512

  for i in range(hp.Int('layers',min_value=0,max_value=3)):
    model.add(keras.layers.Conv2D(hp.Int(f'filter_{i}',min_value=32,max_value=512),(3,3),activation='relu'))

  model.add(keras.layers.MaxPool2D(pool_size=(2,2)))
  model.add(keras.layers.Flatten())

  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))

  if hp.Boolean('dropout'):
    model.add(keras.layers.Dropout(rate=0.25))
  model.add(keras.layers.Dense(5,'softmax'))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

  return model

### Instantiate the Hyperband tuner: Specify the hypermodel, the objective to optimize and the maximum number of epochs to train (max_epochs).

In [ ]:
tuner = Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=5,
                     factor=3,
                     directory='/content/drive/MyDrive/my_dir',
                     project_name='hyper_tuner_01')

INFO:tensorflow:Reloading Oracle from existing project /content/drive/MyDrive/my_dir/hyper_tuner/oracle.json
INFO:tensorflow:Reloading Tuner from /content/drive/MyDrive/my_dir/hyper_tuner/tuner0.json


### A callback to stop training early after reaching a certain value for the validation loss

In [ ]:
stop_early = callbacks.EarlyStopping(monitor='val_loss', patience=5)

### Hyperparameter search: The arguments for the search method are the same as those used for tf.keras.model.fit in addition to the callback above.

In [ ]:
tuner.search(train_set, epochs=2,validation_data=val_set, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 3 Complete [01h 31m 24s]
val_accuracy: 0.5659875869750977

Best val_accuracy So Far: 0.5828166604042053
Total elapsed time: 01h 31m 24s

Search: Running Trial #4

Hyperparameter    |Value             |Best Value So Far 
layers            |3                 |3                 
units             |256               |288               
dropout           |False             |False             
learning_rate     |0.001             |0.001             
units0            |510               |32                
units1            |470               |32                
units2            |244               |32                
units_0           |317               |None              
tuner/epochs      |2                 |2                 
tuner/initial_e...|0                 |0                 
tuner/bracket     |2                 |2                 
tuner/round       |0                 |0                 

Epoch 1/2


### Find the optimal number of epochs to train the model with the hyperparameters obtained from the search.

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 10 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit_generator(train_set, epochs=10, validation_data=val_set)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

### Re-instantiate the hypermodel and train it with the optimal number of epochs from above.

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(train_set, epochs=best_epoch, validation_data=val_set)

### Evaluate the hypermodel on the validation data

In [ ]:
eval_result = hypermodel.evaluate_generator(val_set)
print("[val_loss, val_accuracy]:", eval_result)